In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install tensorflow-metal


ERROR: Could not find a version that satisfies the requirement tensorflow-metal (from versions: none)
ERROR: No matching distribution found for tensorflow-metal


In [4]:
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices()

2.20.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]